# Implement LSH from scratch

In this assignment, you will implement LSH from scratch and predict the labels of the test data. You will then verify the correctness of the your implementation using a "grader" function/cell (provided by us) which will match your implmentation.

The grader fucntion would help you validate the correctness of your code. 

Please submit the final Colab notebook in the classroom ONLY after you have verified your code using the grader function/cell.


**NOTE: DO NOT change the "grader" functions or code snippets written by us.Please add your code in the suggested locations.**

Ethics Code:
1. You are welcome to read up online resources to implement the code. 
2. You can also discuss with your classmates on the implmentation over Slack.
3. But, the code you wirte and submit should be yours ONLY. Your code will be compared against other stduents' code and online code snippets to check for plagiarism. If your code is found to be plagiarised, you will be awarded zero-marks for all assignments, which have a 10% wieghtage in the final marks for this course.

## Reading the data from csv file

In [33]:
# Code to mount google drive in case you are loading the data from your google drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content


In [34]:
# Loading data from csv file
import pandas as pd
data_path = '/content/drive/My Drive/Data_Science_Stuff/Python/Colab_books/Machine_Learning/Assignments/lsh_assignment_data.csv'

df = pd.read_csv(data_path)
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...


In [35]:
# Data Overiview
df['category'].value_counts()

sport            509
business         508
politics         415
tech             399
entertainment    384
Name: category, dtype: int64

### Creating Train and Test Datasets

Note that the labels for test data will not be present in the dataset and hence they are mentioned as NaN.

In [36]:
# The last 10 rows in the csv file are query points, so loading them into test data.
# And loading the reamining points to train_data for which labels are given.

train_data = df.iloc[:-10,:]
test_data = df.iloc[-10:,:]

In [37]:
# For train_data here the labels are in the column named "category".
train_data.shape
print(train_data[train_data.category.isnull()])

Empty DataFrame
Columns: [category, text]
Index: []


In [38]:
test_data

,category,text
2215,NaN,junk e-mails on relentless rise spam traffic i...
2216,NaN,top stars join us tsunami tv show brad pitt r...
2217,NaN,rings of steel combat net attacks gambling is ...
2218,NaN,davies favours gloucester future wales hooker ...
2219,NaN,beijingers fume over parking fees choking traf...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...
2224,NaN,souness delight at euro progress boss graeme s...


## Custom Implementation

### Instructions:

  1. Read in the train_data.
  2. Vectorize train_data using sklearns built in tfidf vectorizer.
  3. Ignore unigrams and make use of both **bigrams & trigrams** and also limit the **max features** to **4000** and **minimum document frequency** to **10**.
  4. After the tfidf vectors are generated as mentioned above, next task is to generate random hyperplanes.
  5. Generate **5 random hyperplanes**. And generate the hyperplanes using a random normal distribution with **mean zero and variance 1**. 
  6. We have set the **numpy random seed to zero**, please do not change it. And then you can make use of **np.random.normal** to generate the vectors for hyperplanes.
  7. As mentioned in the course videos, compute the hash function and also the corresponding hash table for it.
  8. Once the hash table is generated now take in each of the query points from the test data.
  9. Vectorize those query points using the same tfidf vectorizer as mentioned above.
  10. Now use the hash function on this query point and fetch all the similar data points from the hashtable.
  11. Use cosine similarity to compute **11-Nearest Neighbours** from the list of data points obtained in the above step.
  12. Take a majority vote among the 11-Nearest Neighbours and predict the class label for the query point in the test data.
  13. **In case of a tie** in the obtained labels from nearest neighbours, you can pick a label after sorting all the labels **alphabetically**(A-Z), i.e. for example labels starting with A would get more preference than labels starting with Z.
  14. Repeat steps 9 to 13 for all the points in the test data and then finally return a list with all the predicted labels.
  15. Note that there are a total of 10 data points in the test data so the final list you return should be of length 10.
  16. Also note that the cosine similarity function should be written from scratch, you should not directly make use of existing libraries.
  17. Please use the formula of cosine similarity as explained in the course videos, you can make use of numpy or scipy to calculate dot or norm or transpose.

In [53]:
# Please implement this fucntion and write your code wherever asked. Do NOT change the code snippets provided by us.

import numpy as np

def predictLabels (test_data):
  """
  Given the test_data, return the labels for all the rows in the test data.
  Follow the step by step instructions mentioned above.
  """

  np.random.seed(0)

  ##############################################################
  ####  Write YOUR CODE BELOW  as per the above instructions ###
  ##############################################################
  
  #Training data extraction:--
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/
  data_path = '/content/drive/My Drive/Data_Science_Stuff/Python/Colab_books/Machine_Learning/Assignments/lsh_assignment_data.csv'
  train_data = df.iloc[:-10,:]
  train_x=train_data["text"]
  train_y=train_data["category"]

  #Testing data extraction:--
  test_x=test_data["text"]
  test_y=test_data["category"]


  #Tf-Idf vector generating:
  def find_tf_idf(corpus,vectors):
    from sklearn.feature_extraction.text import TfidfTransformer
    from sklearn.feature_extraction.text import TfidfVectorizer

    tf_idf_vect=TfidfVectorizer(ngram_range=(2,3), min_df=10,max_features=4000)#Initiate Tf-idf and create object
    tf_idf_vect.fit(corpus)#Fit over pre_processed corpus
    return tf_idf_vect.transform(vectors)#transform into sparse matrix

  #2: Vectorize using inbuilt function:

  final_tf_idf=find_tf_idf(train_x,train_x)#To find tf_idf vectors
  
  #5: Generate 5 random Hyper planes:
  hyper_planes=[]
  row,dim=(final_tf_idf.shape)
  for i in range(0,5):#Last number is excluded
    hyper_planes.append(np.random.normal(0,1,dim))

  #7 Hash_Function to generate hash_keys:
  def hash_func(vector,hyper_planes):
    
    hash_key=[]
    for j in hyper_planes:
     WX=np.dot(vector,np.array(j).reshape(len(vector),1))[0]# find dot product hyper plane and vector
     if WX<0:#If dot product is negative , encode -1
        hash_key.append(-1)
     else:#If dot product is positive or Zero , encode +1
        hash_key.append(+1)
    hash_key=tuple(hash_key)
    return hash_key


  #7: Generating hash table:
  hash_table={}
  for i in range(row):#Picks each vector from out-put matrix
    hash_key=hash_func(final_tf_idf[i,:].toarray()[0],hyper_planes)
    if hash_key in hash_table.keys():
      hash_table[hash_key].append(i)
    else:
      hash_table[hash_key]=[i]


  #9: Vectorizing query points:
  test_tf_idf=find_tf_idf(train_x,test_x)#To find tf_idf vectors

  #10 Hash values for query points:
  hash_keys_query=[]
  r_test,d_test=(test_tf_idf.shape)
  for i in range(r_test):
    hash_key_q=hash_func(test_tf_idf[i,:].toarray()[0],hyper_planes)#Generate hash_keys
    hash_keys_query.append(hash_key_q)

  #Compute cosine similarity:--
  def cos_sim(vector1,vector2):
    magnitude_1=np.linalg.norm(vector1)
    magnitude_2=np.linalg.norm(vector2)
    return np.dot(vector1,vector2)/(magnitude_1*magnitude_2)
  
  #Return dictionary key for a given dictionary value
  def get_dict_key(dictionary,value):
    key_list=list(dictionary.keys())
    value_list=list(dictionary.values())
    indices=[]
    for v in value_list:#Find indices of the value of interest
      if v==value:
        indices.append(value_list.index(value))

    return [key_list[i] for i in indices]

  #Return key with max values/return key in alphabetical order incase of tie in max values
  def get_major_keys(dictionary):
    values=dictionary.values()
    max_count=max(values)
    return get_dict_key(dictionary,max_count)


  #11 Find 11 nearest neighbors for each query point
  nearest_neighbors={}# Stored in format:keys--> query table row, values-->(neighbor vector row,cosine_sim)

  for query_row,hash_key in enumerate(hash_keys_query):
    if hash_key in hash_table:#find the relevant bucket in hash table
      bucket_vecs=hash_table[hash_key]#find relevant bucket in hash table

      cosine_sim={}#Collect cosine_similarties with all bucket vectors in format:keys-->dataset row, values-->cosine similarities
      for data_row in bucket_vecs:
        cs=cos_sim(final_tf_idf[data_row,:].toarray()[0],test_tf_idf[query_row,:].toarray()[0])
        cosine_sim[data_row]=cs

      #To find top 11 neighbors and their similarities for given query point
      cs_vals=sorted(cosine_sim.values())[:-12:-1]#sort the list in descending order

      for value in cs_vals:
        key=get_dict_key(cosine_sim, value)[0]
        if query_row in nearest_neighbors.keys():
          nearest_neighbors[query_row].append((key,value))
        else:
          nearest_neighbors[query_row]=[(key,value)]

    else:
      print("Point",hash_keys_query.index(i),"cannot fit in hash table!")



  #12 Majority vote amongst 11 nearest neighbors:
  major_class={}#dictionary of test classes in format:keys--> query table row, values-->major class
  for query_row in nearest_neighbors.keys():

    class_count={"sport":0,"business":0,"politics":0,"tech":0,"entertainment":0}#Dictionary of classes
    for data_row,_ in nearest_neighbors[query_row]:#Count classes
      label=train_y.iloc[data_row]
      if label in class_count.keys() :
        class_count[label]=class_count[label]+1
      else:
        print("Label not available!")
  
    #15 To find classes
    major_class[query_row]=get_major_keys(class_count)[0]

  return list(major_class.values())


## Grader Cell

Please execute the following Grader cell to verify the correctness of your above implementation. This cell will print "Success" if your implmentation of the predictLabels() is correct, else, it will print "Failed". Make sure you get a "Success" before you submit the code in the classroom.

In [54]:
###########################################
## GRADER CELL: Do NOT Change this.
# This cell will print "Success" if your implmentation of the predictLabels() is correct and the accuracy obtained is above 80%.
# Else, it will print "Failed"
###########################################
import numpy as np

# Predict the labels using the predictLabels() function
Y_custom = np.array(predictLabels(test_data))

# Reference grader array - DO NOT MODIFY IT
Y_grader = np.array(['tech', 'entertainment', 'tech', 'sport', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport'])

# Calculating accuracy by comparing Y_grader and Y_custom
accuracy = np.sum(Y_grader==Y_custom) * 10

if accuracy >= 80:
  print("******** Success ********","Accuracy Achieved = ", accuracy,'%')
else:
  print("####### Failed #######","Accuracy Achieved = ", accuracy,'%')
  print("\nY_grader = \n\n", Y_grader)
  print("\n","*"*50)
  print("\nY_custom = \n\n", Y_custom)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
******** Success ******** Accuracy Achieved =  90 %
